<a href="https://colab.research.google.com/github/kglobe/python_ml/blob/master/%E6%8E%83%E7%B4%AF%E7%A9%8D%E6%88%90%E4%BA%A4%E9%87%8F%E6%AF%94%E8%BC%83.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
import datetime
import requests
import time
import json
import pytz

def getAllStockTWSE():
  allStockNo = []
  allStockName = {}
  allStockType = {}
  url = "https://api.finmindtrade.com/api/v4/data"
  token = getToken()
  parameter = {
      "dataset": "TaiwanStockInfo",
      "token": token, # 參考登入，獲取金鑰
  }
  resp = requests.get(url, params=parameter)
  data = resp.json()
  data = pd.DataFrame(data["data"])
  data = data.loc[data['type']=='twse']
  try:
    for index, row in data.iterrows():
      if len(row['stock_id']) == 4:
        allStockNo.append(row['stock_id'])
        allStockName[row['stock_id']] = row['stock_name']
        allStockType[row['stock_id']] = row['industry_category']
  except Exception as e:
    pass
  return allStockNo,allStockName,allStockType

def getAllStockTPEX():
  allStockNo = []
  allStockName = {}
  allStockType = {}
  url = "https://api.finmindtrade.com/api/v4/data"
  token = getToken()
  parameter = {
      "dataset": "TaiwanStockInfo",
      "token": token, # 參考登入，獲取金鑰
  }
  resp = requests.get(url, params=parameter)
  data = resp.json()
  data = pd.DataFrame(data["data"])
  data = data.loc[data['type']=='tpex']
  try:
    for index, row in data.iterrows():
      if len(row['stock_id']) == 4:
        allStockNo.append(row['stock_id'])
        allStockName[row['stock_id']] = row['stock_name']
        allStockType[row['stock_id']] = row['industry_category']
  except Exception as e:
    pass
  return allStockNo,allStockName,allStockType

def getToken():
  url = "https://api.finmindtrade.com/api/v4/login"
  parload = {
      "user_id": "kglobe",
      "password": "ft06270417",
  }
  data = requests.post(url, data=parload)
  data = data.json()
  return data['token']

def getAvgTradeVol(stockNo,startDate,endDate):
  data = getStockDataByFinMind(stockNo,startDate,endDate)
  avg = data[['成交股數']].mean(axis=0)
  return avg['成交股數']

def getStockDataByFinMindAll(startDate):
  url = "https://api.finmindtrade.com/api/v4/data"
  parameter = {
      "dataset": "TaiwanStockPrice",
      "start_date": startDate,
      "token": token, # 參考登入，獲取金鑰
  }
  resp = requests.get(url, params=parameter)
  data = resp.json()
  data = pd.DataFrame(data["data"])
  if data.empty == False:
    print('取得['+startDate+']股票資料')
    data.columns = ['日期','股票代碼','成交股數','成交金額','開盤價','最高價','最低價','收盤價','漲跌價差','成交筆數']
  return data

def getStockDataByFinMind(stockNo,startDate,endDate):
  url = "https://api.finmindtrade.com/api/v4/data"
  parameter = {
      "dataset": "TaiwanStockPrice",
      "data_id": stockNo,
      "start_date": startDate,
      "end_date": endDate,
      "token": token, # 參考登入，獲取金鑰
  }
  resp = requests.get(url, params=parameter)
  data = resp.json()
  data = pd.DataFrame(data["data"])
  data.columns = ['日期','股票代碼','成交股數','成交金額','開盤價','最高價','最低價','收盤價','漲跌價差','成交筆數']
  data = data.set_index('日期')
  data = data.drop(['股票代碼'], axis=1)
  print(stockNo+'共有'+str(len(data.index))+'天的資料')
  return data

token = getToken()
# 欲爬取的股票代碼
twse_stock_list = ['1418','1475','1452','1451','1472','1447','1449','1466','1413','2615','5007','1459','1467','1457','1417','1455','2027','1460','2606','1476','2008','2029','1477','2025','2609','2603','1470','2017','1439','2633','1440','1445','2069','5608','9958','2023','2024','2028','1464','2636','2031','1463','2014','1409','1232','1419','2006','2020','2012','2634','2002','2611','2010','1213','1441','1474','4414','2022','1210','2034','2618','1414','1443','2617','4438','2610','2015','1454','2009','2007','1218','1434','1444','1473','5607','2032','1219','1220','2030','1702','1410','1225','1737','1402','2211','1233','1423','1234','2613','1216','2033','1215','1231','8367','1468','2642','1201','2612','2038','2608','2630','2607']
txpe_stock_list = []

now = datetime.datetime.now(pytz.timezone('Asia/Taipei'))
starttime = now-datetime.timedelta(days=7)
endtime = now-datetime.timedelta(days=1)
rundate = starttime
allstock = pd.DataFrame()
while rundate <= endtime:
  df = getStockDataByFinMindAll(rundate.strftime('%Y-%m-%d'))
  if df.empty == False:
    allstock = pd.concat([allstock,df])
  rundate = rundate + datetime.timedelta(days=1)

# startdate = starttime.strftime('%Y-%m-%d')
# enddate = endtime.strftime('%Y-%m-%d')
# print('startdate:',startdate)
# print('enddate:',enddate)
all_avg_vol_arr = []

# for stock in twse_stock_list:
#   rlt = getAvgTradeVol(stock,startdate,enddate)
#   all_avg_vol_arr.append([stock,round(rlt/1000, 0)])

# for stock in txpe_stock_list:
#   rlt = getAvgTradeVol(stock,startdate,enddate)
#   all_avg_vol_arr.append([stock,round(rlt/1000, 0)])

for stock in twse_stock_list:
  rlt = (allstock[allstock['股票代碼']==stock]['成交股數']/1000).mean(axis=0)
  all_avg_vol_arr.append([stock,round(rlt, 0)])

for stock in txpe_stock_list:
  rlt = (allstock[allstock['股票代碼']==stock]['成交股數']/1000).mean(axis=0)
  all_avg_vol_arr.append([stock,round(rlt, 0)])

all_avg_vol = pd.DataFrame(np.array(all_avg_vol_arr),columns=['股票代號', '平均交易量'])
print(all_avg_vol)

取得[2021-05-04]股票資料
取得[2021-05-05]股票資料
取得[2021-05-06]股票資料
取得[2021-05-07]股票資料
取得[2021-05-10]股票資料
     股票代號    平均交易量
0    1418     33.0
1    1475     64.0
2    1452   7504.0
3    1451   1195.0
4    1472    163.0
..    ...      ...
97   2612   7456.0
98   2038  42981.0
99   2608    244.0
100  2630    748.0
101  2607  27629.0

[102 rows x 2 columns]


In [2]:
from IPython.display import display, clear_output, HTML
from urllib.request import urlopen
import pandas as pd
import datetime
import requests
import sched
import time
import json
import pytz

s = sched.scheduler(time.time, time.sleep)

def mergeResult(result, result_shift):
  try:
    result_merge = pd.merge(result, result_shift, left_index=True, right_index=True)
    result_merge = result_merge.dropna()
  except Exception as e:
    print('mergeResult Exception:',e)
  return result_merge

def sendMsg(msg):
  URL = 'https://notify-api.line.me/api/notify'
  headers = {'Authorization': 'Bearer ' + '2wNtEnwpoBvIPHUqagBT5WT3ItzpDlDMLjSsa8BZzwK',}
  payload = {'message': msg}

  r = requests.post(URL, headers = headers, params = payload)
  if r.status_code == 200:
    print('send to LINE ok!')
  else:
    print('sendto LINE error!',r.text)

def tableColor(val):
    if val >= 0.5:
        color = 'red'
    elif val < 0.5:
        color = 'green'
    else:
        color = 'white'
    return 'color: %s' % color

def removeArrayEle(arr,elem):
  try:
    arr.remove(elem)
  except Exception as e:
    pass
  return arr

def stock_crawler(tse_targets,txpe_targets):
  clear_output(wait=True)
      
  # 組成stock_list
  tse_stock_list = '|'.join('tse_{}.tw'.format(target) for target in tse_targets)
  txpe_stock_list = '|'.join('otc_{}.tw'.format(target) for target in txpe_targets)

  #　query data
  query_url = "http://mis.twse.com.tw/stock/api/getStockInfo.jsp?ex_ch="+ tse_stock_list+'|'+txpe_stock_list
  print('query_url:',query_url)
  # data = json.loads(urlopen(query_url).read())
  resp = requests.get(query_url)
  data = resp.json()

  # 過濾出有用到的欄位
  columns = ['c','n','z','tv','v','o','h','l','y']
  df = pd.DataFrame(data['msgArray'], columns=columns)
  df.columns = ['股票代號','公司簡稱','當盤成交價','當盤成交量','累積成交量','開盤價','最高價','最低價','昨收價']
  df = df.merge(all_avg_vol, left_on='股票代號', right_on='股票代號')
  df = df.replace('-',0)
  df['累積成交量'] = df['累積成交量'].astype(float)
  df['平均交易量'] = df['平均交易量'].astype(float)
  df['累積成交量百分比'] = df['累積成交量']/df['平均交易量']
  
  # df.insert(9, "累積成交量百分比", 0.0)
  # 新增漲跌百分比
  # for x in range(len(df.index)):
  #   if df['累積成交量'].iloc[x] != '-':
  #       df.iloc[x, [2,3,4,5,6,7,8,9]] = df.iloc[x, [2,3,4,5,6,7,8,9]].astype(float)
  #       df['累積成交量百分比'].iloc[x] = df['累積成交量'].iloc[x]/df['平均交易量'].iloc[x]

  # 紀錄更新時間
  now = datetime.datetime.now(pytz.timezone('Asia/Taipei'))
  print("更新時間:" + now.strftime('%Y-%m-%d %H:%M:%S'))

  # show table
  df_show = df.style.applymap(tableColor, subset=['累積成交量百分比'])
  display(df_show)

  compare_time1 = datetime.datetime.strptime(str(now.date())+'9:00+0800', '%Y-%m-%d%H:%M%z')
  compare_time2 = datetime.datetime.strptime(str(now.date())+'9:05+0800', '%Y-%m-%d%H:%M%z')
  if now > compare_time1 and now < compare_time2 :
    over_df = df[df['累積成交量百分比']>0.5]
    for index, row in over_df.iterrows():
      msg = '\n'+row['股票代號']+'_'+row['公司簡稱']+'\n累積成交量比例為：'+str(row['累積成交量百分比'])
      print(msg)
      sendMsg(msg)
      tse_targets = removeArrayEle(tse_targets,row['股票代號'])
      txpe_targets = removeArrayEle(txpe_targets,row['股票代號'])
  else:
    over_df = df[df['累積成交量百分比']>2]
    for index, row in over_df.iterrows():
      msg = '\n'+row['股票代號']+'_'+row['公司簡稱']+'\n累積成交量比例為：'+str(row['累積成交量百分比'])
      print(msg)
      sendMsg(msg)
      tse_targets = removeArrayEle(tse_targets,row['股票代號'])
      txpe_targets = removeArrayEle(txpe_targets,row['股票代號'])

  start_time = datetime.datetime.strptime(str(now.date())+'00:00+0800', '%Y-%m-%d%H:%M%z')
  end_time =  datetime.datetime.strptime(str(now.date())+'23:30+0800', '%Y-%m-%d%H:%M%z')
  # 判斷爬蟲終止條件
  if now >= start_time and now <= end_time:
      s.enter(5, 0, stock_crawler, argument=(tse_targets,txpe_targets,))

# 每秒定時器
s.enter(5, 0, stock_crawler, argument=(twse_stock_list,txpe_stock_list,))
s.run()

query_url: http://mis.twse.com.tw/stock/api/getStockInfo.jsp?ex_ch=tse_1418.tw|tse_1475.tw|tse_1452.tw|tse_1451.tw|tse_1472.tw|tse_1447.tw|tse_1449.tw|tse_1466.tw|tse_1413.tw|tse_2615.tw|tse_5007.tw|tse_1459.tw|tse_1467.tw|tse_1457.tw|tse_1417.tw|tse_1455.tw|tse_2027.tw|tse_1460.tw|tse_2606.tw|tse_1476.tw|tse_2008.tw|tse_2029.tw|tse_1477.tw|tse_2025.tw|tse_2609.tw|tse_2603.tw|tse_1470.tw|tse_2017.tw|tse_1439.tw|tse_2633.tw|tse_1440.tw|tse_1445.tw|tse_2069.tw|tse_5608.tw|tse_9958.tw|tse_2023.tw|tse_2024.tw|tse_2028.tw|tse_1464.tw|tse_2636.tw|tse_2031.tw|tse_1463.tw|tse_2014.tw|tse_1409.tw|tse_1232.tw|tse_1419.tw|tse_2006.tw|tse_2020.tw|tse_2012.tw|tse_2634.tw|tse_2002.tw|tse_2611.tw|tse_2010.tw|tse_1213.tw|tse_1441.tw|tse_1474.tw|tse_4414.tw|tse_2022.tw|tse_1210.tw|tse_2034.tw|tse_2618.tw|tse_1414.tw|tse_1443.tw|tse_2617.tw|tse_4438.tw|tse_2610.tw|tse_2015.tw|tse_1454.tw|tse_2009.tw|tse_2007.tw|tse_1218.tw|tse_1434.tw|tse_1444.tw|tse_1473.tw|tse_5607.tw|tse_2032.tw|tse_1219.tw|tse_1220.

,股票代號,公司簡稱,當盤成交價,當盤成交量,累積成交量,開盤價,最高價,最低價,昨收價,平均交易量,累積成交量百分比
0,1418,東華,0,0,21.000000,11.7000,11.7000,11.1500,11.4500,37.000000,0.567568
1,1475,業旺,48.1000,1,46.000000,49.4000,50.4000,47.0000,50.0000,91.000000,0.505495
2,1452,宏益,25.5000,42,8296.000000,25.4500,25.5000,24.2500,23.2000,9019.000000,0.919836
3,1451,年興,22.3500,27,3178.000000,20.8000,22.3500,20.7500,20.3500,796.000000,3.992462
4,1472,三洋紡,11.0000,12,246.000000,10.2000,11.0000,10.2000,10.1000,141.000000,1.744681
5,1447,力鵬,14.4000,1882,49000.000000,14.2500,14.7500,13.8500,14.1000,57396.000000,0.853718
6,1449,佳和,20.4500,5,426.000000,19.7500,20.7000,19.7500,19.7000,419.000000,1.016706
7,1466,聚隆,18.1000,89,2221.000000,17.2500,18.2500,17.2000,16.9500,5164.000000,0.430093
8,1413,宏洲,0,0,3132.000000,12.6000,13.0000,12.2000,11.8500,3824.000000,0.819038
9,2615,萬海,101.5000,2256,63930.000000,95.3000,101.5000,93.5000,92.3000,63378.000000,1.008710



1451_年興
累積成交量比例為：3.992462311557789
send to LINE ok!

1457_宜進
累積成交量比例為：2.040641897784755
send to LINE ok!

1476_儒鴻
累積成交量比例為：2.490675241157556
send to LINE ok!

2025_千興
累積成交量比例為：2.266849315068493
send to LINE ok!

2017_官田鋼
累積成交量比例為：5.942894507410636
send to LINE ok!

9958_世紀鋼
累積成交量比例為：2.617430960222954
send to LINE ok!

1419_新紡
累積成交量比例為：3.247422680412371
send to LINE ok!

2020_美亞
累積成交量比例為：3.04780206581792
send to LINE ok!

2010_春源
累積成交量比例為：2.041616356757627
send to LINE ok!

2007_燁興
累積成交量比例為：3.091379905627805
send to LINE ok!

5607_遠雄港
累積成交量比例為：2.177570093457944
send to LINE ok!

2032_新鋼
累積成交量比例為：2.298324478953821
send to LINE ok!

2211_長榮鋼
累積成交量比例為：2.4932773109243698
send to LINE ok!
